# CZ4041 Group 28 Notebook 3
To Run this Notebook, Import the .csv Files from the Notebook 2<br>
The .csv Files are:

- train_missing_count.csv 
- test_missing_count.csv

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import tensorflow as tf

from keras import backend as K
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from keras.optimizers import Adam, Nadam
from keras.preprocessing.image import ImageDataGenerator

from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
step2_train = pd.read_csv("../input/stage2/train_missing_count.csv").sort_values(by=["ImageId"])

# Reset Index, as Sorting Values by ImageId Messes Up Index
step2_train.reset_index(inplace = True, drop = True)
# Replace the .png extension to .jpg
step2_train["ImageId"] = step2_train["ImageId"].str.replace("png","jpg")

step2_train.head()

In [ ]:
train_df = pd.read_csv("../input/severstal-steel-defect-detection/train.csv").sort_values(by=['ImageId'])
train_df.head()

In [ ]:
step2_train.info()
print()
train_df.info()

# Creating Dataframes for EDA
We will First Create the final_train_df. 

final_train_df will be the Main Dataframe Used/Manipulated from now on.

In [ ]:
# Lets Merge the train_df and step2_train
final_train_df = train_df.merge(step2_train, on='ImageId', how='right').sort_values(by='ImageId').reset_index(drop=True)
final_train_df

# For Downloading final_train_df csv, if needed.
# final_train_df.to_csv('final_train_df.csv', index=False)

In [ ]:
# Notice that here is 12997-429 = 12568 (Total Number of Images)
# 5902 + 6239 + 425 + 2 = 12568, and
# 5902 + 6239 + 425*2 + 2*3 = 12997 (Total Number of Entries)

data =  final_train_df["ClassId"].groupby(final_train_df["ImageId"]).count()
plt.figure(figsize=(5,7))
plt.suptitle("Number of Defect Classes per Image")
plt.grid()

ax = sns.countplot(data)
ax.set_ylabel("Count")
ax.set_xlabel("No. of Defects")

print("Numeric info : \n")
print("0 Defects: ",len(data[data == 0]))
print("1 Defect : ",len(data[data == 1]))
print("2 Defects: ",len(data[data == 2]))
print("3 Defects: ",len(data[data == 3]))
print("4 Defects: ",len(data[data == 4]))

It is necessary to note that at this point of the pipeline, images that are **not defects but has been classified as a defect** do not have a ClassId tied to them. 

As such, the EDA we just performed only utilizes images with a ClassId.

Next, we shall investigate what value shall be our threshold for the allMissing scoring system we implemented. An allMissing score below the threshold value will be treated as a defective image.

The ideal threshold score will maximize the correct predictions for defects, and minimize misclassification of non-defect images. The three usable metrics to be considered are:

F1 Score = $ 2*(Recall * Precision) \over (Recall + Precision)$

Recall = $ TP \over (TP+FN)$

Precision = $ TP \over (TP + FP)$

In [ ]:
final_train_df.loc[final_train_df['ClassId'].notnull()]

In [ ]:
final_train_df.loc[final_train_df['ClassId'].isnull()]

In [ ]:
def getnondefect(threshold):
    return (len(final_train_df.loc[(final_train_df['ClassId'].isnull()) & (final_train_df['allMissing'] > threshold)])/5902)

def getdefect(threshold):
    return (len(final_train_df.loc[(final_train_df['ClassId'].notnull()) & (final_train_df['allMissing'] < threshold)])/7095)

def TP(threshold):
    return (len(final_train_df.loc[(final_train_df['ClassId'].notnull()) & (final_train_df['allMissing'] < threshold)])/7095)*100

def FN(threshold):
    return (len(final_train_df.loc[(final_train_df['ClassId'].notnull()) & (final_train_df['allMissing'] > threshold)])/7095)*100

def FP(threshold):
    return (len(final_train_df.loc[(final_train_df['ClassId'].isnull()) & (final_train_df['allMissing'] < threshold)])/7095)*100

def TN(threshold):
    return (len(final_train_df.loc[(final_train_df['ClassId'].isnull()) & (final_train_df['allMissing'] > threshold)])/5902)*100

def recall(TP, FN):
    return (TP/(TP+FN))

def precision(TP, FP):
    return TP/(TP+FP)

def F1(Recall, Precision):
    return (2*(Recall*Precision))/(Recall+Precision)

def accuracy(TP, TN, FP, FN):
    return (TP+TN)/(TP+TN+FP+FN)


In [ ]:
# Creating a Dataframe to Compare Threshold Value to Non-Defect Images (% of them Below Threshold Value)

threshold_setting = pd.DataFrame(float(i/1000) for i in range(0, 1001))
threshold_setting = threshold_setting.drop([0,1000])

threshold_setting['isnull'] = threshold_setting[0].apply(getnondefect)
threshold_setting['Defective Images (%) Below Threshold'] = threshold_setting[0].apply(getdefect)

threshold_setting['TP'] = threshold_setting[0].apply(TP)
threshold_setting['TN'] = threshold_setting[0].apply(TN)
threshold_setting['FN'] = threshold_setting[0].apply(FN)
threshold_setting['FP'] = threshold_setting[0].apply(FP)

threshold_setting['Accuracy'] = threshold_setting.apply(lambda x: accuracy(x['TP'], x['TN'], x['FP'], x['FN']), axis = 1)
threshold_setting['Recall'] = threshold_setting.apply(lambda x: recall(x['TP'], x['FN']), axis = 1)
threshold_setting['Precision'] = threshold_setting.apply(lambda x: precision(x['TP'], x['FP']), axis = 1)
threshold_setting['F1'] = threshold_setting.apply(lambda x: F1(x['Recall'], x['Precision']), axis = 1)

# Renaming Column names
threshold_setting.columns = ["Threshold Value", "Non-Defect Images (%) Above Threshold", 'Defective Images (%) Below Threshold',
                            "TP", "TN", "FN", "FP", "Accuracy", "Recall", "Precision", "F1"]
# threshold_setting = threshold_setting.drop([0, 1000])

threshold_setting

In [ ]:
fig = px.line(threshold_setting)

fig.add_scatter(x=threshold_setting["Threshold Value"], y=threshold_setting['Non-Defect Images (%) Above Threshold'], name="Non-Defect Images (%) Above Threshold")
fig.add_scatter(x=threshold_setting["Threshold Value"], y=threshold_setting['Defective Images (%) Below Threshold'], name="Defective Images (%) Below Threshold")

fig.update_layout(title = "% Defective/Non Defective Images",
                 xaxis_title = "Threshold Value"
                 )

fig.show()

Plotting this graph allows us to see that perhaps at the intersect of both line would be our ideal threshold value. This intersect marks the threshold value that yields the highest combined number of defect and non-defect classification.

However,our pipeline should punish False Positives as images classified as defective despite being non-defective are heavily penalized in the competition. Therefore we should consider using the Precision to determine our threshold value.

Recall = $ TP \over (TP+FP)$


In [ ]:
# Lets Visualize How the Behaviours of our Various Metrics with Regards to the Threshold Value
fig = px.line(threshold_setting)

fig.add_scatter(x=threshold_setting["Threshold Value"], y=threshold_setting['Accuracy'], name="Accuracy")
fig.add_scatter(x=threshold_setting["Threshold Value"], y=threshold_setting['Precision'], name="Precision")
fig.add_scatter(x=threshold_setting["Threshold Value"], y=threshold_setting['Recall'], name="Recall")
fig.add_scatter(x=threshold_setting["Threshold Value"], y=threshold_setting['F1'], name="F1")

fig.update_layout(title = "Accuracy, Recall, Precision, F1 vs Threshold Value",
                 xaxis_title = "Threshold Value"
                 )

fig.show()

It is necessary to highlight that the Precision alone would be a tough metric to use, as the graph shows that maximizing Precision will result in a threshold value close to 0, and all images are considered defects. Also, the Accuracy, Precision and F1 are at their minimum when threshold value is set to 0.

As such, we will have to use a metric that combines both Recall and Precision. Since the F1 Score utilizes both Precision and Recall, we shall investigate the use of F1 score as the metric to decide on the threshold value.

F1 Score = $ 2*(Recall * Precision) \over (Recall + Precision)$


In [ ]:
# Here we Find the Row with the Highest F1 Score Attained. Included in the Row would be the Various Metrics.
F1 = list(threshold_setting['F1'])
F1.sort()
threshold_setting.loc[threshold_setting['F1'] == F1[-1]]

In [ ]:
# The Threshold Value where Accuracy, Recall, Precision and F1 are almost the Same (with reference to the graph)

threshold_setting.loc[abs(threshold_setting['Accuracy']-threshold_setting['Recall'])
                      + abs(threshold_setting['Recall']-threshold_setting['Precision'])
                      + abs(threshold_setting['Precision']-threshold_setting['F1'])<0.003]

We decided to use the threshold value of 0.919 as it resulted in the most consistent score for all four metrics.

In [ ]:
# We Now can Apply the Threshold Score, 0.919 to our Dataset
THRESHOLD = 0.919
class1 = final_train_df.loc[(final_train_df['ClassId'] == 1) & (final_train_df['allMissing'] < THRESHOLD)]
class2 = final_train_df.loc[(final_train_df['ClassId'] == 2) & (final_train_df['allMissing'] < THRESHOLD)]
class3 = final_train_df.loc[(final_train_df['ClassId'] == 3) & (final_train_df['allMissing'] < THRESHOLD)]
class4 = final_train_df.loc[(final_train_df['ClassId'] == 4) & (final_train_df['allMissing'] < THRESHOLD)]

print("Class 1 Defects: " + str(len(final_train_df.loc[(final_train_df['ClassId'] == 1)]))
     + "\nClass 2 Defects: " + str(len(final_train_df.loc[(final_train_df['ClassId'] == 2)]))
     + "\nClass 3 Defects: " + str(len(final_train_df.loc[(final_train_df['ClassId'] == 3)]))
     + "\nClass 4 Defects: " + str(len(final_train_df.loc[(final_train_df['ClassId'] == 4)]))
      
     + "\n\nClass 1 Defects Predicted: " + str(len(class1))
     + "\nClass 2 Defects Predicted: " + str(len(class2))
     + "\nClass 3 Defects Predicted: " + str(len(class3))
     + "\nClass 4 Defects Predicted: " + str(len(class4))
     )